In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, BatchNormalization, MaxPooling2D, AveragePooling2D
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam

# Load dataset from CSV files
dataframeINPUT = pd.read_csv('Inputs_Sparse.csv')
dataframeOUTPUT = pd.read_csv('Outputs_Full.csv')
X, Y = dataframeINPUT.to_numpy(), dataframeOUTPUT.to_numpy()

# Visualize the data
plt.plot(Y)
plt.xlabel("Number of Data Arrays")
plt.ylabel("Data Point Value")
plt.show()

In [ ]:
# Split the dataset into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.5)

# Display dataset shapes
print("Input Shape:", X.shape)
print("Output Shape:", Y.shape)
in_dim = X.shape[1]
out_dim = Y.shape[1]
print("Output Dimension:", out_dim)

# Reshape the input data to fit the CNN model
xtrain = xtrain.reshape(xtrain.shape[0], in_dim, 1)
xtest = xtest.reshape(xtest.shape[0], in_dim, 1)

# Display reshaped input shape
print("Reshaped Input Shape:", xtrain.shape)

In [ ]:
# Create the CNN model
model = Sequential()
model.add(Conv2D(160, kernel_size=3, activation="relu", input_shape=(in_dim, 1)))
model.add(Conv2D(320, kernel_size=3, activation="relu"))
model.add(AveragePooling2D())
model.add(Flatten())
model.add(Dropout(0.1))
model.add(Dense(out_dim))

# Compile the model
model.compile(loss="mae", optimizer=Adam(), metrics=['accuracy'])
model.summary()

# Train the model
history = model.fit(xtrain, ytrain, validation_data=(xtest, ytest), epochs=10, batch_size=100)


In [ ]:
# Predict on the test set
ypred = model.predict(xtest)

# Save predictions to a CSV file
np.savetxt("Predictions.csv", ypred, delimiter=",")

# Display shapes for verification
print("Test Input Shape:", xtest.shape)
print("Test Output Shape:", ytest.shape)
print("Predicted Output Shape:", ypred.shape)

In [ ]:
# Display training and validation accuracy
plt.plot(history.history['accuracy'], 'g', label='Training accuracy')
plt.plot(history.history['val_accuracy'], 'b', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Display training and test loss
plt.plot(history.history['loss'], 'r--', label='Training Loss')
plt.plot(history.history['val_loss'], 'b-', label='Test Loss')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

# Visualize a sample prediction along with actual and input data
sample_index = 150
predicted_sample = np.reshape(ypred[sample_index, :], (24, 108))
input_sample = np.reshape(xtest[sample_index, :], (in_dim, 1))
actual_sample = np.reshape(ytest[sample_index, :], (24, 108))

fig, (ax1, ax2, ax3) = plt.subplots(3, 1)
ax1.contour(np.flipud(predicted_sample))
ax1.axis('equal')
ax2.contour(np.flipud(input_sample))
ax3.contour(np.flipud(actual_sample))
ax3.axis('equal')